In [1]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import matplotlib.patches as patches
import matplotlib.path as path
from itertools import chain
from operator import add

from numpy.linalg import inv
import scipy.linalg as la
import numpy.random as rnd

# shapely and descartes must be installed separately.
from shapely.geometry import Point, Polygon, LineString
import shapely.ops as ops
import descartes

# This is Andy's hand-cooked kernel module. 
# Probably better to replace with something like GPy.
import kernels as kn
reload(kn)
#%matplotlib inline

<module 'kernels' from 'kernels.pyc'>

In [2]:
# These are some helper functions
def inequalityMatrices(verts):
    """
    Convert a polygon stored as a collection of vertices (assuming listed in order)
    to a constraint of the form
    Ax <= B
    """
    
    n = len(verts)
    
    A = np.zeros((n,2))
    b = np.zeros(n)
    
    for k in range(n):
        M = np.hstack((np.zeros((2,2)),np.ones((2,1))))
        M[0,:2] = verts[k]
        M[1,:2] = verts[(k+1) % n]
        U,S,Vh = la.svd(M)
        V2 = Vh[-1]
        vec = np.hstack((verts[(k+2) % n],1))
        sgn = 1. if np.dot(V2,vec) < 0 else -1.
        A[k,:] = sgn * V2[:2]
        b[k] = -sgn * V2[-1]
        
    return A,b
    
    

def isCollision(g,constraintList):
    x = g[:2]
    for  A,b in constraintList:
        res = b - np.dot(A,x)
        if res.min() >= 0:
            return True
    return False

def rangeFinderValue(g,constraintList):
    """
    Given g = (x,y,theta), this finds the point on the nearest polygon that the laser range-finder would hit.
    
    """
    x = g[:2]
    theta = g[-1]
    TList = np.zeros(len(constraintList))
    for k in range(len(constraintList)):
        A,b = constraintList[k]
        a = np.dot(A,np.array([np.cos(theta),np.sin(theta)]))
        bCur = b - np.dot(A,x)
        NegInd = a < 0
        PosInd = a >= 0
        
        # This will have issues if the beam is exactly parallel to an edge
        
        tLow = np.max(bCur[NegInd]/a[NegInd])
        tHigh = np.min(bCur[PosInd]/a[PosInd])
        
        t = tLow if (tLow <= tHigh) and (tLow >=0) else np.inf
        
        TList[k] = t
        
        d = TList.min()
        
    return d


def carPolygon(g):
    cL = 2.
    cW = 1.
    
    x = g[:2]
    theta = g[-1]
    
    R = np.array([[np.cos(theta),-np.sin(theta)],
                  [np.sin(theta),np.cos(theta)]])
    
    carPoly = np.array([[-cL/2,-cW/2],
                        [cL/2,0],
                        [-cL/2,cW/2]])
    
    carPoly = np.dot(carPoly,R.T) + np.outer(np.ones(3),x)
    
    return Polygon(carPoly)


def sort_zeros(arr):
    pos=0
    duplicate=np.zeros((len(arr)))
    for i in range(0, len(arr)):
        if np.any(arr[i]):
            duplicate[pos] = arr[i]
            pos += 1    
    return duplicate

def EKF_propagate(x_hat_plus,P_plus,w_m,v_m,dt):
    
    diagonal=np.matrix([[0.01*0.01, 0],
                       [0, 0.01*0.04]]).reshape(2,2)
    
    x_hat_min = np.empty([len(x_hat_plus)]).reshape((len(x_hat_plus), 1))
    x_hat_min[0:3]=x_hat_plus[0:3]+dt*np.array([[v_m*np.cos(x_hat_plus[2,0]),
                                                 v_m*np.sin(x_hat_plus[2,0]),
                                                                       w_m]]).reshape((3, 1))
    x_hat_min[3:]=x_hat_plus[3:]
    #Jacobian

    Phi = np.matrix([[1, 0, -dt*v_m*np.sin(x_hat_plus[2,0]),
                      0, 1,  dt*v_m*np.cos(x_hat_plus[2,0]),
                      0, 0, 1                                ]]).reshape((3, 3))
    G = -dt*np.matrix([[np.cos(x_hat_plus[2,0]), 0,
                       np.sin(x_hat_plus[2,0]), 0,
                                         0, 1]]).reshape((3, 2))
    P_min=np.zeros((len(P_plus),len(P_plus)))
    P_min[0:3,0:3]=Phi*P_plus[0:3,0:3]*Phi.transpose()+G*v_m**2*diagonal*G.transpose()
    P_min= (P_min+P_min.transpose())/2
    if len(P_plus) > 3:
        P_min[0:3,3:]=Phi*P_plus[0:3,3:]
        P_min[3:,0:3]=P_min[0:3,3:].transpose()
        P_min[3:,3:]=P_plus[3:,3:]
    
    P_min= (P_min+P_min.transpose())/2
    return  x_hat_min,P_min        

def EKF_update(x_hat_min,P_min,gamma_match,gamma_newlm,sigma_d,sigma_th,d,x_lm,d_max):
    R=np.matrix([[sigma_d*sigma_d, 0],
                [0, sigma_th*sigma_th]]).reshape(2,2)


    N_lm=x_lm.size/2
    fd=np.array([d])

    #measurement for each landmark at each instant

    z_d=np.zeros((N_lm))
    z_th=np.zeros((N_lm))

    for j in range(0,N_lm):
        z_d[j]=np.sqrt( (x_lm[0,j] - x_hat_min[0])**2 + (x_lm[1,j] - x_hat_min[1])**2 )+sigma_d*np.random.normal(0, 1)
        z_th[j]=np.arctan2(x_lm[1,j]-x_hat_min[1],x_lm[0,j] - x_hat_min[0])+sigma_th*np.random.normal(0, 1)#* 180 / np.pi

    for j in range(0,N_lm):
        if z_d[j]>d_max:
            z_d[j]=0
            z_th[j]=0
        
    z_d=sort_zeros(z_d)
    z_th=sort_zeros(z_th)
    z_d=np.trim_zeros(z_d)
    z_th=np.trim_zeros(z_th)

    z_dist=z_d
    z_theta=z_th
    n_m=(len(x_hat_min)-3)/2

    
    J=np.matrix([[0, -1],
                [1,  0]]).reshape(2,2)

    if not np.any(z_dist):
        x_hat_plus = x_hat_min
        P_plus = P_min
        minmd=np.zeros((0,1))
        minij=np.zeros((0,1))
        mdij=np.zeros((n_m,1))
        posi=np.asmatrix(np.zeros((3,1))).reshape(3,1)
        res=[]
        diagS=[]
        pri_l=[]
    else:
        
        n_z=len(z_dist)
        z_p=np.asmatrix(np.zeros((2,n_z)))
        Rpi=np.zeros((2,2,n_z))
        posi=np.asmatrix(np.zeros((3,n_z))).reshape(3,n_z)
        res=[]
        diagS=[]
        pri_l=[]
        for x in range(1, n_z+1):
            z_p[:,x-1]=z_dist[x-1]*np.matrix([np.cos(z_theta[x-1]),np.sin(z_theta[x-1])]).transpose()
            G=np.matrix([[np.cos(z_theta[x-1]), -z_dist[x-1]*np.sin(z_theta[x-1])],
                     [np.sin(z_theta[x-1]),  z_dist[x-1]*np.cos(z_theta[x-1])]]).reshape(2,2)
            Rpi[:,:,x-1]=G*R*G.transpose()


        for i in range(1, n_z+1):
            C=np.matrix([[np.cos(x_hat_min.item(2)), -np.sin(x_hat_min.item(2)),
                      np.sin(x_hat_min.item(2)),  np.cos(x_hat_min.item(2))]]).reshape(2,2)
            rij=np.asmatrix(np.zeros((2,n_m)))
            Hj=np.zeros((2,len(x_hat_min),n_m))
            Sij_inv=np.zeros((2,2,n_m))
            mdij=np.zeros((n_m,1))
        
            for j in range(1, n_m+1):
                S=x_hat_min[2+2*j-1:2+2*j+1]-x_hat_min[0:2]
                z_hat_j=np.asarray(C.T*S)
                Hj[:,0:3,j-1] = (-C).T*np.hstack((np.eye(2),J*S))
                Hj[:,2+2*j-1:2+2*j+1,j-1]=C.T
                rij[:,j-1]=z_p[:,i-1]-z_hat_j
                Sij_inv[:,:,j-1]=la.inv(Hj[:,:,j-1]*P_min*Hj[:,:,j-1].T+Rpi[:,:,i-1])
                mdij[j-1]=rij[:,j-1].T*np.asmatrix(Sij_inv[:,:,j-1])*rij[:,j-1]
        
            if not mdij.any():
                minmd=np.zeros((0,1))
                minij=np.zeros((0,1))
            else:
                minmd=mdij.min()
                minij=mdij.argmin()
        
            
            if not minmd.any() or minmd> gamma_newlm:
            #print "New landmark"
            
                p_newlm = x_hat_min[0:2]+C*z_p[:,i-1].reshape(2,1)
                x_hat_min=np.vstack((x_hat_min,p_newlm))
                fif=x_hat_min[0]+z_dist[i-1]*np.cos(z_theta[i-1])
                afa=x_hat_min[1]+z_dist[i-1]*np.sin(z_theta[i-1])
                posi[:,i-1]=np.asmatrix(np.vstack((fif,afa,fd)))
                #posi=np.vstack((p_newlm,d))
                HR=-C.T*np.hstack((np.eye(2),J*(p_newlm-x_hat_min[0:2])))
                HLi=C.T
                P_LL=HLi.T*(HR*P_min[0:3,0:3]*HR.T+Rpi[:,:,i-1]*HLi)
                P_xL=-P_min[:,0:3]*HR.T*HLi
                top=np.hstack((P_min,P_xL))
                bottom=np.hstack((P_xL.T,P_LL))
                P_min=np.vstack((top,bottom))
            
            
            elif minmd < gamma_match:
                K=P_min*Hj[:,:,minij].T*Sij_inv[:,:,minij]
                x_hat_min+K*rij[:,minij].reshape(2,1)
                P_min=(np.eye(len(P_min))-K*Hj[:,:,minij])*P_min*(np.eye(len(P_min))-K*Hj[:,:,minij]).T+K*Rpi[:,:,i-1]*K.T
                res=np.append(res,rij[:,minij].reshape(2,1))
                diagS=np.append(diagS,np.diagonal(la.inv(Sij_inv[:,:,minij])).reshape(2,1))
                posi[:,i-1]=np.asmatrix(np.vstack((0,0,0)))
                
        
            x_hat_plus=x_hat_min
            P_plus=(P_min+P_min.T)/2
    return x_hat_plus,P_plus,res,diagS,minmd,mdij,minij,posi

In [3]:
# Assume that vertices are enumerated counter-clockwise
# Build the constraints

h=0
leftPoly = [[-8,-8],[-7,-8],[-7,8],[-8,8]]
botPoly = [[-7,-8],[7,-8],[7,-7],[-7,-7]]
rightPoly = [[7,-8],[8,-8],[8,8],[7,8]]
topPoly = [[-7,7],[7,7],[7,8],[-7,8]]

# Number of obstacles
NumObs = 8


obsPoly = []

for obs in range(NumObs):
                 
    NumVert = rnd.randint(3,8)#Number of Vertices
                 
    Cen = 14. * rnd.rand(2) - 7.#Center of Polygon, eg - array([-6.50255021, -1.95047639])
    
    Theta = np.sort(2*np.pi * rnd.rand(NumVert))#Creates N Corner Angles, and np sorts them in ascending orderd  = rnd.rand(NumVert)#d is length of sides(assumption) 
    
    obsVert = np.outer(np.ones(NumVert),Cen) +  np.vstack((np.cos(Theta),np.sin(Theta))).T
    #adds center x co-ordinate with cos(theta) and y co-ordinate with sin(theta)
    
    obsPoly.append(obsVert)

shapeVerticesList = [leftPoly,botPoly,rightPoly,topPoly]

polyList = [Polygon(verts) for verts in shapeVerticesList]
#function Polygon creates polygons with vertices
#verts is the verts of each polygon

polyUnion = ops.unary_union(polyList)

obsList = [Polygon(verts) for verts in obsPoly]
obsUnion = ops.unary_union(obsList)

#for obs in obsList:
  #  ax.add_patch(descartes.PolygonPatch(obs,fc = 'blue',alpha=0.5))

constraintList = [inequalityMatrices(verts) for verts in shapeVerticesList]#+obsPoly]
#ax.axis('equal')

In [4]:
dt=1
N=250;
#true velocity. Changing this profile allows changing the trajectory
v_true=0.1*np.ones(N);

#true rotational velocity
w_true =0.03 *np.ones(N)

#Radius of circle the robot goes on
r_R=v_true[0]/w_true[0] 

#Robot sensor range

d_max = 0.4*r_R; #Maximum sensor range
d_min = 0.1; 
                      
#Noise Strength
sigma_d = 0.1; # [m]    distance measurements
sigma_th= 0.04;# rad    bearing measurements
sigma_v = sigma_d*v_true;
sigma_w = sigma_th*w_true;

x_true=np.zeros(shape=(3,N))
x_true[:,0]=np.array([0,-3,0])

for pos in range(1,N):
    
    x_true[0,pos] = x_true[0,pos-1]+ v_true[pos-1]*dt*np.cos(x_true[2,pos-1]);#x
    x_true[1,pos]= x_true[1,pos-1]+ v_true[pos-1]*dt*np.sin(x_true[2,pos-1]);#y
    x_true[2,pos]= x_true[2,pos-1]+ w_true[pos-1]*dt;#phi

v_m=v_true+sigma_v*np.random.normal(0, 1, N)
w_m=w_true+sigma_w*np.random.normal(0, 1, N )

x_s=x_true[0,:]
y_s=x_true[1,:]
ang=x_true[2,:]
rob = np.row_stack((x_s, y_s))

#LandMarks

#x_hat_plus=np.array([0+8,-3+8,0]).reshape(3,1)

#LandMark Co-ordinates
#LandMark Co-ordinates
#x_lm=np.array([[0,0,-8.5,8.5,-7,-7,7,7],[-4,12,3,3,-2,8,-2,8]])
x_lm=np.array([[0,0,4,-4,3,-3,3,-3],[-4,4,0,0,-3,-3,3,3]])
N_lm=x_lm.size/2
#x_lm=np.array([[3],[4]])
#ax.plot(x_lm[0,:],x_lm[1,:],'b*

In [5]:
# Bookkeeping

# pre-allocate space for certain values we want to keep track of

xR_hat_min = np.asmatrix(np.zeros((3,N))); # robot state estimate after Propagation
xR_hat_plus = np.asmatrix(np.zeros((3,N)));   # robot state estimate after update
PR_min =np.zeros((3,3,N));      #covariance matrices of the Robot after Propagation
PR_plus = np.zeros((3,3,N));     # covariance matrices of the Robot after Update

res = np.zeros((2,N*N_lm));      # measurement residuals, stacked in chronological order
diagS = np.zeros((2,N*N_lm)); 
x_hat_1=np.array([0,-3,0])                         # diagonal of residual covariance in chronological order
#land_pos=np.matrix(np.zeros((3,N)))
land_pos = np.matrix(np.zeros((3))).reshape(3,1)
resind = 1;
flag=0;

xR_hat_plus[:,0] = x_hat_1.reshape((3, 1)) 
PR_plus[:,:,0] = np.zeros((3,3));

In [6]:
x_hat_plus=np.array([0,-3,0]).reshape((3, 1)) 
P_plus=np.zeros((3,3))
gamma_match = 2;#could be 4
gamma_newlm = 300;#range15-50
#Extended Kalman Filter
for i in range(1,N):
    x_hat_min,P_min=EKF_propagate(x_hat_plus,P_plus,w_m[i-1],v_m[i-1],dt) 
    xR_hat_min[:,i]=x_hat_min[0:3].reshape(3,1)
    PR_min[:,:,i] = P_min[0:3,0:3]
    P_min= np.asmatrix(P_min)
    x_hat_plus,P_plus,res,diagS,minmd,mdij,minij,posi=EKF_update(x_hat_min,P_min,gamma_match,gamma_newlm,sigma_d,sigma_th,i,x_lm,d_max)
    #land_pos[:,i]=posi
    land_pos=np.hstack((land_pos,posi))
    
    xR_hat_plus[:,i]=x_hat_plus[0:3].reshape(3,1)
    PR_plus[:,:,i] = P_plus[0:3,0:3]
    
x=xR_hat_plus[0,:]
x=x.tolist()
x=list(chain.from_iterable(x))

y=xR_hat_plus[1,:]
y=y.tolist()
y=list(chain.from_iterable(y))

yaw=xR_hat_plus[2,:]
yaw=yaw.tolist()
yaw=list(chain.from_iterable(yaw))

In [7]:
fig = plt.figure()
ax = fig.add_subplot(111)
fig.set_dpi(100)
fig.set_size_inches(10, 10)
ax.add_patch(descartes.PolygonPatch(polyUnion,fc = 'blue',alpha=0.5))
ax.plot(x_lm[0,:],x_lm[1,:],'b+')
ax.plot(x_true[0,:],x_true[1,:])
ax.set_xlim(-8, 8)
ax.set_ylim(-8, 8)

patch = patches.Rectangle((0, 0), 0, 0, fc='y')
line =  patches.Rectangle((0, 0), 0, 0, fc='r')
def init():
    ax.add_patch(patch)
    ax.add_patch(line)
    return patch,line,

def animate(i):
    x_wall = np.array([xR_hat_plus[0,i],xR_hat_plus[1,i]])
    theta = xR_hat_plus[2,i]
    g = np.hstack((x_wall,theta))
    t = rangeFinderValue(g,constraintList)
    xInt = x_wall + t*np.array([np.cos(theta),np.sin(theta)])
    x_Int=xInt.tolist()
    ax.plot(xR_hat_plus[0,i],xR_hat_plus[1,i],'y.')
    ax.plot(xInt[0],xInt[1],'r.')
    for j in range(1,len(land_pos.T)-1):
        if land_pos[2,j]==i and i!=0:
            ax.plot(land_pos[0,j],land_pos[1,j],'r*')
    #line.set_width(t)
    #line.set_height(0.05)
    #line.set_xy([x[i], y[i]])
    #line._angle = np.rad2deg(yaw[i])
    patch.set_width(0.8)
    patch.set_height(0.4)
    patch.set_xy([x[i], y[i]])
    patch._angle = np.rad2deg(yaw[i])
    return patch,line,

anim = animation.FuncAnimation(fig, animate,
                               init_func=init,
                               frames=len(x),
                               interval=500,
                               blit=True)
HTML(anim.to_html5_video())

In [8]:
thetaWall = 1.
NumSamp = N

gk = kn.mvGaussian(theta=.1)

x_s=xR_hat_plus[0,:]
y_s=xR_hat_plus[1,:]
ang=xR_hat_plus[2,:]
rob = np.asarray(np.row_stack((x_s, y_s)))

x = np.zeros(2)
Theta = 2. * np.pi * rnd.rand(NumSamp)
#Theta = ang

SampPerBeam = 6

XList = []
YList= []

D = np.zeros(NumSamp)
for k in range(NumSamp):
    x = rob[:,k]
    theta = Theta[k]
    g = np.hstack((x,theta))
    d = rangeFinderValue(g,constraintList)
        
    D[k] = d
    if (d > 1e-3) and (not isCollision(g,constraintList)):
        XSamp = np.zeros((SampPerBeam,2))
        u = np.array([np.cos(theta),np.sin(theta)])
        dVec = np.zeros(SampPerBeam)
        dVec[0:2] = [d,0]
        if SampPerBeam > 2:
            dVec[2:] = d * rnd.rand(SampPerBeam-2)
            

        for samp in range(SampPerBeam):
            XSamp[samp] = x + dVec[samp] * u
        
        YSamp = np.tanh(-thetaWall * (d-dVec))
        XList.append(XSamp)
        YList.append(YSamp)
        
    if d < 0:
        print 'Inside'
        

X = np.vstack(XList)
Y = np.hstack(YList)

K = gk.k(X,X)+1e-8 * np.eye(len(X))
Emat = la.solve(K,Y,sym_pos=True)

X1Grid,X2Grid = np.meshgrid(np.linspace(-8,8,100),np.linspace(-8,8,100))

XGrid = np.vstack((X1Grid.flatten(),X2Grid.flatten())).T

kGrid = gk.k(XGrid,X)

YGrid = np.dot(kGrid,Emat).reshape(X1Grid.shape)

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)

ax.contour(X1Grid,X2Grid,YGrid,[0],linewidths=2)
# ax.plot(X[:,0],X[:,1],'g.',alpha=.1)

#for xl in XList:
     #plt.plot(xl[:2,0],xl[:2,1],'k',alpha=.3)
     #plt.plot(xl[0,0],xl[0,1],'r*',alpha=.3)
    #plt.plot(xl[1,0],xl[1,1],'go',alpha=.3)
    
ax.add_patch(descartes.PolygonPatch(polyUnion,fc = 'red',alpha=0.1))

#for obs in obsList:
   # ax.add_patch(descartes.PolygonPatch(obs,fc = 'red',alpha=0.1))

ax.set_xticks([])
ax.set_yticks([])
ax.axis('equal')
plt.savefig('gpMap_SimpleSlam.pdf',bbox_inches='tight',transparent=True)